# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [ ]:
#%pip install azure-ai-projects
#%pip install yfinance

In [ ]:
import glob
import json
import ipyplot
import matplotlib.pyplot as plt
import os
import requests
import sys
import time
import yfinance as yf

#from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient 
from azure.identity import DefaultAzureCredential
#from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from azure.ai.agents.models import (
    FunctionTool,
    ListSortOrder,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
)
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Any, Callable, Set, Dict, List, Optional
from PIL import Image
from pathlib import Path

from user_functions import user_functions, azuremaps_weather

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

## 1. Agent definition - Weather forecasts

## Connect to AI Foundry Project ### Project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


### Testing the Azure Maps Weather function

We will the Azure Maps service to provide weather information.

In [6]:
weather = azuremaps_weather("Toronto")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-06-09T09:17:00-04:00',
    'phrase': 'Cloudy',
    'iconCode': 7,
    'hasPrecipitation': False,
    'isDayTime': True,
    'temperature': {'value': 16.1, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 18.3, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 15.8, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 87,
    'dewPoint': {'value': 13.9, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 90.0, 'localizedDescription': 'E'},
     'speed': {'value': 7.4, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 7.4, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 2,
    'uvIndexPhrase': 'Low',
    'visibility': {'value': 12.9, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 91,
    'ceiling': {'value': 1707.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1006.5, 'unit': 'mb', 'unitType': 14},
    'pressureTendenc

### Agent

In [8]:
model = "gpt-4o-mini"
name = "gpt4o-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

# user functions to use
user_functions: Set[Callable[..., Any]] = {
    azuremaps_weather,
}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# We need to define this option to the project (not in the documentation ?)
project_client.enable_auto_function_calls(toolset=toolset)

# Agent creation
agent = project_client.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    tool_resources=toolset,
)

print(f"Created agent with ID = {agent.id}")

TypeError: AgentsClient.enable_auto_function_calls() got an unexpected keyword argument 'toolset'. Did you mean 'tools'?

### Tests

In [ ]:
prompt = "Hello, generate a full report for the weather in Paris today"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

In [ ]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

In [ ]:
run.usage

In [ ]:
messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

### Other examples

In [ ]:
prompt = "Hello, generate a full report for the weather in London today. Print the results in English and French."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

In [ ]:
prompt = "Hello, print the UV informations for London today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

In [ ]:
prompt = "Hello, What is the temperature in Paris? Display the time of the forecast"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

In [ ]:
prompt = "Hello, What is the temperature and UV information for Marseille? Display the time of the forecast for the UTC zone"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

In [ ]:
prompt = "Generate a weather report for Paris. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)

### Post processing

In [ ]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    if agents.data[i].name == name:
        print(f"Deleting agent {agents.data[i].id}")
        project_client.agents.delete_agent(agents.data[i].id)